<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width="140px" alt="EPFL_logo">

## Image Processing Laboratory Notebooks
---

This Jupyter Notebook is part of a series of computer laboratories that are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-511 Image Processing I**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2024.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Yan Liu](mailto:yan.liu@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).
    
---
# Lab 3: Morphology
**Released**: Thursday, December 12, 2024

**Submission deadline**: Monday, December 23, 2024, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=522)

**Grade weight**: Lab 3 (17 points), 10% of the overall grade

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## Imports
In this first cell we import the required Python libraries:
* [`matplotlib.pyplot`](https://matplotlib.org), to display images
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays
* [`cv2`](https://docs.opencv.org/master/), for image processing in Python

We will then load the `ImageViewer` class. For more information on it, you can either see the complete documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer), run the Python command `help(viewer)` after loading the class, or refer to [Lab 0: Introduction](../0_Introductory_lab/Introductory.ipynb)).

Finally, we load the images you will use in the exercise to test your algorithms. 

In [ ]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import required packages for this lab
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import cv2 as cv
from interactive_kit import imviewer as viewer 

# Load images to be used in this lab 
plate = cv.imread('images/plate.tif', cv.IMREAD_UNCHANGED)
butterfly = cv.imread('images/butterfly-graylevel.tif', cv.IMREAD_UNCHANGED)

In the following cell we import the JavaScript `ImageAccess` class, created specifically for this course, which facilitates the creation and modification of images. You can find the full documentation of the class [here](https://biomedical-imaging-group.github.io/image-access/).

In [ ]:
%use javascript
// import image-access as Image
var Image = require('image-access')

# Morphology Implementations (12 points)

In the first part of this laboratory, you will learn

1. how to implement several morphological filters in a low-level language (like JavaScript), and
2. how to apply them in image processing applications using [`OpenCV`](https://opencv.org/).

We will focus mainly on 2D gray-level morphology, but keep in mind that the same operations can easily be adapted to color images by treating each color channel as an independent gray-level image.

⚠️ **Note: Each cell that contains code begins with `%use sos` or `%use javascript`. This indicates if the code in this specific cell should be written in Python or JavaScript. Do not change or remove any lines of code that begin with an `%`. They need to be on the first line of each cell!**

# 1. Structuring elements (2 points)

As you have seen in the course, a structuring element (often abbreviated with `strel` in the code) can be described by a binary image consisting of a foreground and a background, which are usually represented with `true` (or $1$) and `false` (or $0$), respectively. They are used in morphological filters such as dilation, erosion, opening, closing, and all others that are derived from those. The purpose of the structuring element is to define the shape and size used by a morphological filter. **Your task in this section is to implement 2 different structuring elements, namely a cross and a disc.**

**For 1 point each**, implement the functions `cross(n)` and `disc(n)` to build the shapes of the structuring elements of size $n \times n$. The function `square(n)` is provided and serves as a reference. The "disc" shape contains elements that are inside a circle tangent to the frame of the $n \times n$ square, and the "cross" shape contains a horizontal and a vertical line that meet in the center of the square, both with a single-pixel width.

The images below show an example of how the different elements should look with a size of $9 \times 9$.

| ![Structuring elements](images/Structuring_elements_showcase.png) |
|:--:| 
| *Examples of how a correct $9\times 9$ cross, disc and square structuring elements look like.* |


⚠️ **Note: The proposed structuring elements are all symmetric and ***you only need to implement for odd $n$***, it is not necessary to reflect the structuring element as we would do in a generic implementation. However, you can still do it if you want.**

## 1.A. Square structuring element

The next cell provides the example function `square(n)`, which returns a square of size $n \times n$ filled with $1$s.

In [ ]:
%use javascript
// function that takes as input an integer n and returns an n x n image of '1's
function square(n){ 
    // declare the output image
    var output = new Image(n, n);
    // iterate through each pixel
    for(var x = 0; x < n; x++){  
        for(var y = 0; y < n; y++){
            // assign pixel value at location (x,y) to '1'
            output.setPixel(x, y, 1);
        }
    }
    return output;
}

The next cell runs the function `square(n)` and stores the result in the variable `strel_square` which is converted to Python to display it in another cell. Feel free to experiment with the size passed to the function and observe the results.
   
⚠️ **Note: The method `.toArray()` is needed to convert the variable to Python.**

In [ ]:
%use javascript
%put strel_square
// runs the function you implemented above and converts the Image object to an array
// feel free to change the size passed to this function and observe the result
var strel_square = square(9).toArray();

Run the next cell to use Python to display the structuring element as an image.
    
⚠️ **Note: Throughout the lab, we will reuse the following general cell structure:**

- **Function/Code,**
- **Running the function,**
- **Display/evaluate the results**

**We will not give a detailed description every time and we will let you do more and more on your own as we progress.**

In [ ]:
%use sos
# Display the binary image with a title and numerated pixel grid
plt.close('all')
disp_square = viewer(np.array(strel_square), title=f'Square structuring element of size {np.shape(strel_square)}', clip_range=[0, 1], axis=True, pixel_grid=True, cmap='viridis')

When working in Python, we can use the [OpenCV](https://docs.opencv.org/master/) library to generate structuring elements and perform image processing tasks.

To use OpenCV (called `cv2` in Python for historical reasons) it is best practice to import it as `cv`, which has already been done in the first cell of the notebook. When using the morphological filters provided by OpenCV, the structuring element that should be passed to the functions is a `numpy` array. We can either define it ourselves using NumPy, for example, a $9 \times 9$ square as in the cell below.

In [ ]:
%use sos
strel_square_custom = np.ones((9,9))

or we can use [`cv.getStructuringElement()`](https://docs.opencv.org/trunk/d4/d86/group__imgproc__filter.html#gac342a1bb6eabf6f55c803b09268e36dc), which takes as input parameters:
* `shape`: [`cv.MORPH_RECT`](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gac2db39b56866583a95a5680313c314ad) to generate a rectangle, [`cv.MORPH_ELLIPSE`](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gac2db39b56866583a95a5680313c314ad) to generate an ellipse, [`cv.MORPH_CROSS`](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gac2db39b56866583a95a5680313c314ad) to generate a cross, and
* `ksize`: The shape of the structuring element, e.g. `(9, 9)`

You can look at the documentation for a full description of the function. Run the next cell to get the same $9 \times 9$ square we defined above.

In [ ]:
%use sos
strel_square_cv = cv.getStructuringElement(cv.MORPH_RECT, ksize=(9,9))

We can compare the three structuring elements (JS, NumPy, OpenCV). We can do this either visually by providing the `ImageViewer` class with a list of images and titles, as in the cell below.

⚠️ **Note: If you changed the size of the JS structuring element, change it back to $9$ and rerun for the following comparisons. If the right-most image is only partially visible, make sure to close the file browser tab on the left by clicking on the folder symbol or try zooming out the webpage.**

In [ ]:
%use sos

# Close existing figures to release memory
plt.close('all')
# Display the three structuring elemnts side by side to compare them
disp_square = viewer([np.array(strel_square), strel_square_custom, strel_square_cv], title=['JavaScript', 'Custom numpy array', 'OpenCV'], 
                     subplots=(1,3), clip_range=[0, 1], cmap='viridis', axis=True, pixel_grid=True)

or numerically, to make sure all the implementations provide the same result by using the `assert` statement, as in the cell below.

**Note: [`np.allclose(arr1, arr2)`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html) returns `True` if `arr1` and `arr2` are equal to a certain tolerance.**

In [ ]:
%use sos
# Compare the JS version to the custom numpy array by subtracting one from the other and then counting the number of non-zero pixels, which should be 0
assert np.allclose(strel_square, strel_square_custom), 'The JS and NumPy version do not agree.'
# Compare the JS version to the OpenCV version similarly
assert np.allclose(strel_square, strel_square_cv), 'The JS and OpenCV version do not agree.'
print('Indeed, the three structuring elements are exactly the same.')

## 1.B. Cross structuring element

Now it's your turn! In the cell below, **for 1 point**, complete the code in JavaScript to implement the `cross(n)` function.

💡 *Hint: If you need to, you can use the function `parseInt(x)` to get the largest integer smaller than a given $x\in\mathbb{R}$. Remember that your implementation only needs to work for odd `n` (this is true for the whole section).*

In [ ]:
%use javascript

// function that takes as input an integer n and returns the cross structuring element of size (n x n)
function cross(n){ 
    // declare output image
    var output = new Image(n, n);
    
    // YOUR CODE HERE
    
    return output;
}

Now run the following $2$ cells to generate a $9\times 9$ structuring element and visualize it in Python. Feel free to change the size in the next cell.

In [ ]:
%use javascript
%put strel_cross
// runs the function you implemented above and converts the Image object to an array
// feel free to change the size passed to this function and observe the result
var strel_cross = cross(9).toArray();

In [ ]:
%use sos
# Display the binary image with a title and numbered pixel grid
plt.close('all')
disp_cross = viewer(np.array(strel_cross), title=f'Cross structuring element of size {np.shape(strel_cross)}', 
                    axis=True, pixel_grid=True, cmap='viridis', clip_range=[0, 1])

In Python, we have several ways of generating a cross structuring element. One, using only NumPy is to generate an array of zeros and use advanced indexing to set the middle row/column to $1$. However, OpenCV also offers us a straightforward way by passing `cv.MORPH_CROSS` to `getStructuringElement`. Run the cell below to create and display the OpenCV cross. You can also change the size of the structuring element and see the result.

In [ ]:
%use sos
# Generate the cross structuring element with OpenCV - feel free to play with the size passed to this function
strel_cross_cv = cv.getStructuringElement(cv.MORPH_CROSS, ksize=(9, 9))

# Display it
plt.close('all')
disp_cross_cv = viewer(strel_cross_cv, title=f'Python cross of size {np.shape(strel_cross_cv)}', 
                       axis=True, pixel_grid=True, cmap='viridis', clip_range=[0, 1])

The following cell tests if the Python and JavaScript structuring elements are identical. To make sure you pass the test, **verify that both are of the same size**!

⚠️ **Note: Throughout this section, we will give you the freedom to choose the size of the JS and the OpenCV structuring element. However, if you do change it, make sure to change it back to the original for the following comparisons. This is true for every exercise. When you hand in your notebook, all comparison cells should run without any errors.**

In [ ]:
%use javascript
%get strel_cross_cv
// Make a small test on the size of the structuring elements
if(Image.arrayCompare(Image.shape(strel_cross), Image.shape(strel_cross_cv)) == false){
    console.log('WARNING!\nThe size of the two structuring elements is not the same:\nstrel_cross = (' + Image.shape(strel_cross) + '), strel_cross_cv = (' + Image.shape(strel_cross_cv) + ')\n');
} 
// Now on their pixel-wise equality
if(Image.arrayCompare(strel_cross, strel_cross_cv) == false){
    console.log('WARNING!\nThe two structuring elements are not the same. Look at the difference between the two images above and try to find what you are doing wrong.\n');
}else{
    // If everything is ok, print a victory message
    console.log("Yes! The crosses are identical.");}

## 1.C. Disc structuring element 

In the cell below, **for 1 point**, complete the code in JavaScript to implement the `disc(n)` function.

In [ ]:
%use javascript

// function that takes as input an integer n and returns the disc structuring element of size (n x n)
function disc(n){ 
    // Ddclare output image
    var output = new Image(n, n);
    
    // YOUR CODE HERE
    
    return output;
}

In [ ]:
%use javascript
%put strel_disc
// runs the function you implemented above and converts the Image object to an array
// feel free to change the size passed to this function and observe the result
var strel_disc = disc(9).toArray();

In [ ]:
%use sos
# Display the binary image with a title and numerated pixel grid
plt.close('all')
disp_disc = viewer(np.array(strel_disc), title=f'Disc structuring element of size {np.shape(strel_disc)}', 
                   axis=True, pixel_grid=True, cmap='viridis', clip_range=[0, 1])

Unfortunately, OpenCV doesn't provide a disc structuring element. However, it provides elliptical structuring elements by passing `cv.MORPH_ELLIPSE` to `getStructuringElement`. Usually, an ellipse that has the same height and width should be just a normal circle/disc, so let's see what happens if we generate such a disc using OpenCV's ellipse generator. Run the cell below to generate an OpenCV interpretation of a disc.

In [ ]:
%use sos
# Generate an ellipse with height = width
strel_ellipse = cv.getStructuringElement(cv.MORPH_ELLIPSE, ksize=(9,9))
# Display the result
plt.close('all')
disp_ellipse = viewer(strel_ellipse, title='OpenCV\'s definition of a disc', pixel_grid=True, axis=True, cmap='viridis')

Does your JavaScript implementation look different from the *disc* in the cell above? If so, don't worry, you didn't do anything wrong. As you can hopefully see, this isn't a circular disc, but still some kind of ellipse, even though the height and width are set to the same value. Either this is a bug in the implementation of `getStructuringElement()` or they simply have another interpretation of a circle (which is wrong). Anyhow, the lesson here is, do not trust anything blindly. This means we need to create our own function in Python to generate a disc structuring element. Luckily for you, we have already done this.

**Note: Because it is very inefficient to use for loops in Python (and you should only do it if necessary!), this task was implemented using a [`lambda` function](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) and the NumPy function [`np.fromfunction`](https://numpy.org/doc/stable/reference/generated/numpy.fromfunction.html).**
    
For the scope of this lab, you do not need to understand the `lambda` function. However, we do recommend you to go through the documentation and completely understand the following cell -- it will improve your programming skills!

Run the next cell to declare the function `disc(n)`.

In [ ]:
%use sos
# Function that generates a disc structuring element in python
def disc(n):
    # Define the function of a circle as a lambda function
    circle_func = lambda i, j: ((i - n//2)**2 + (j - n//2)**2) <= (n//2)**2
    # Set all elements of the array that are inside the circle of diameter n to 1 - np.uint8 to match the type used by OpenCV for structuring elements
    output = np.fromfunction(circle_func, shape=(n,n)).astype(np.uint8)
    # Return the structuring element
    return output

Now run the next one to declare a circular structuring element. Feel free to change `n`, and look at the effect.

In [ ]:
%use sos
# Here we generate a disc structuring element with python by calling the function implemented above
strel_disc_python = disc(9)
# And display it
plt.close('all')
disp_disc = viewer(strel_disc_python, title=f'Python disc of size {np.shape(strel_disc_python)}', 
                   pixel_grid=True, axis=True, cmap='viridis')

Now that we have a working function to compare your implementation to, lets do it! Run the cell below to test your implementation in JS.

In [ ]:
%use javascript
%get strel_disc_python
// This cell tests if the two structuring elements (JavaScript and Python) are identical, which they should be
if(Image.arrayCompare(Image.shape(strel_disc), Image.shape(strel_disc_python)) == false){
    console.log('WARNING!\nThe size of the two structuring elements is not the same:\nstrel_cross = (' + Image.shape(strel_disc) + '), strel_cross_cv = (' + Image.shape(strel_disc_python) + ')\n');
}
if(Image.arrayCompare(strel_disc, strel_disc_python) == false){
    console.log('WARNING!\nThe two structuring elements are not the same. Look at the difference between the two images above and try to find what you are doing wrong.\n');
}else{
    // If they are, we print a victory message
    console.log("Well done! Your disc is better than that of OpenCV.");}

# 2. Debugging (1 point)

This section is the introduction to implementing morphological operators. The provided function `erodeBug` (given in the next cell) is supposed to perform an erosion operation on an image, using a square structuring element of size $3 \times 3$. While **all the JavaScript syntax is correct, there are two bugs in the implementation of `erodeBug`**. **For 1 point**, inspect the code below, fix the two bugs, and explore the cells below to run the function on the images `plate` and `butterfly` to see the result. If you have trouble finding the bugs, it might be a good idea to look at the incorrect output of the function first.

**Note:**
- **`img.nx` and `img.ny` can be used to get the dimensions of `img`,**
- **`img.getNbh(x, y, w, h)` returns the neighbourhood of size $(w\times h)$ around the location $(x,y)$ of `img`,**
- **`Math.min(a,b)` calculates the minimum between $a$ and $b$.**

In [ ]:
%use javascript

// function that erodes a structure with a (3 x 3) square. Original contains two bugs.
function erodeBug(img){ 
    // the structuring element b should be a 3x3 square
    var b = square(3);

    // loop through every pixel of the image
    for(var x = 0; x < img.ny; x++){
        for(var y = 0; y < img.ny; y++){
            // extract the 3x3 neighbourhood around pixel (x,y)
            var neigh = img.getNbh(x, y, 3, 3);
            // initializing the minimum value to the largest number possible in JS
            var valmin = Number.MAX_VALUE;
            // loop through every pixel of the neighborhood
            for(var k = 0; k < 3; k++){
                for(var l = 0; l < 3; l++){
                    // check if the structuring element is either 'true' or '1' at the pixel location
                    if(b.getPixel(k, l) == true || b.getPixel(k, l) == 1){
                        // calculate new minimum value
                        valmin = Math.min(neigh.getPixel(k, l), valmin);
                    }
                }
            }
            // set the pixel at location (x,y) to the calculated minimum value
            img.setPixel(x, y, valmin);
        }
    }    
    
    return img;
}

Run the next cell to apply the function `erodeBug()` to both images and put the result in Python.

In [ ]:
%use javascript
%get plate
%get butterfly
%put eroded_plate
%put eroded_butterfly

// convert the images to Image objects
var plate_img = new Image(plate);
var butterfly_img = new Image(butterfly);
// run erodeBug for the plate image
var eroded_plate = erodeBug(plate_img).toArray()
// run erodeBug for the butterfly image
var eroded_butterfly = erodeBug(butterfly_img).toArray()

Run the next cell to see the result of your version of `erodeBug`, as well as the difference between the original and your result. You should be able to test visually whether you fixed the bugs succesfully. 

In [ ]:
%use sos
# Defining the images and their titles
images = [plate, np.array(eroded_plate),  butterfly, np.array(eroded_butterfly)]
titles = ['Original plate', 'Eroded plate', 'Original butterfly', 'Eroded butterfly']
# Close all previous figures to release memory
plt.close('all')
# Display the images with their titles (you can pass lists of images and titles as arguments)
disp_erodeBug = viewer(images, title=titles, subplots=(2,2))

If you're unsure about the result, it's always a good idea to test a function on an input to which we know the output. For example, we know that if we erode a rectangle of height 3 surrounded by zeros with a $3 \times 3$ square, the result should be a black image with a white horizontal line, right? So let's try it... Run the cells below to create the test image and apply the function `erodeBug` on it.

In [ ]:
%use sos
%put square_image --to javascript
# To define the image with a square in the middle, we initialize a 9x9 image of zeros
square_image = np.zeros((5, 11))
# and insert the rectangle of 1s
square_image[1:4, 1:10] = 1
# Let's see how it looks
img_square_image = viewer(square_image, title='Rectangle of height 3 sorrounded by zeros', pixel_grid=True, axis=True, subplots=(1,1))

In [ ]:
%use javascript
%put eroded_square
// run the function erodeBug on the square image
var eroded_square = erodeBug(new Image(square_image)).toArray();

As mentioned above, the result should be a horizontal line of **height 1** and **width 7**, in an otherwise black image. If this isn't the case, there might still be a bug in the `erodeBug` function above.

In [ ]:
%use sos
# Display the result of the eroded square
plt.close('all')
img_eroded_square = viewer(np.array(eroded_square), title='erodeBug() on rectangle image', pixel_grid=True, axis=True, clip_range=[0, 1], subplots=(1,1))

# 3. Morphological filters (9 points)

In this part, you are asked to implement the morphological filters given in the table below. Click on their names for a quick link to where you have to implement them.

| $\text{Filter}$ | $\text{Function}$ | $\text{Definition / Mathematical notation}$   |
|------------|---------------|----------------------------------------------------------|
| [Erosion](#3.A.-Erosion)              (3.A) | `erosion()`   | $f \ominus b$                                            |
| [Dilation](#3.B.-Dilation)            (3.B) | `dilation()`  | $f \oplus b$                                             |
| [Median](#3.C.-Median-filter)         (3.C) | `median()`    | $\mathrm{MED}(f, b)$                                     |
| [Open](#3.D.-Opening)                 (3.D) | `open()`      | $f \circ b = (f \ominus b) \oplus b$                     |
| [Close](#3.E.-Closing)                (3.E) | `close()`     | $f \bullet b = (f \oplus b) \ominus b$                   |
| [Gradient](#3.F.-Gradient-filter)     (3.F) | `gradient()`  | $\bigtriangledown (f, b) = (f \oplus b) - (f \ominus b)$ |
| [Top-hat](#3.G.-Top-hat-filter)       (3.G) | `topHat()`    | $\mathrm{TH}(f, b) = f - (f \circ b)$                    |
| [Bottom-hat](#3.H.-Bottom-hat-filter) (3.H) | `bottomHat()` | $\mathrm{BH}(f, b) = (f \bullet b) - f$                  |

Each function is worth **1 point**. You will start by implementing the most basic morphological filters: `erosion()`, `dilation()`, and `median()` in JavaScript and compare them to the OpenCV equivalents. The rest will be implemented in Python. The results of your implementation will be shown on the images `plate` and `butterfly`.

## 3.A. Erosion

In the cell below, **for 1 point**, complete the code in JavaScript to implement the `erosion(img, b)`.
    
⚠️ **Note: We hope that you looked in detail at the function `erodeBug`, as you can use it as a hint on how to solve the exercises of this section. Note the main difference between your correct version of `erodeBug` and the function `erosion` is: `erosion` takes as input parameter the structuring element `b`!**

Remember from your course notes, given an image $f$ with support $\Omega_f$ and a structuring element $b$ with support $\Omega_b$, we define erosion as

$$
    (f \ominus b)[\mathbf{k}] = \min_{\mathbf{q}\in\Omega_b}\left\lbrace f\left[\mathbf{k} + \mathbf{q}\right] \mid (\mathbf{k}+\mathbf{q})\in\Omega_f \right\rbrace\,.
$$
 
As we mentioned before, because all the structuring elements are symmetric and $N$ is odd, you need not worry about reflecting them. See the note in [1. Structuring elements](#1.-Structuring-elements-(2-points)).
    
💡 *Hint: You can use `Number.MAX_VALUE` to get the highest number possible in JavaScript.*

In [ ]:
%use javascript

// function that performs an erosion on the image 'img' using the structuring element 'b'
function erosion(img, b){
    // declaring the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    
    return output;
}

// here we declare the structuring element
var b = square(5); // Feel free to change it to your liking (using the functions implemented in part 1) and observe the results.

Run the next cell to apply the `erosion()` function to the images `plate` and `butterfly`.

In [ ]:
%use javascript
%put plate_erosion
%put butterfly_erosion

// running the operation and converting the images back to python
var plate_erosion = erosion(new Image(plate), b).toArray();
var butterfly_erosion = erosion(new Image(butterfly), b).toArray();

Run the next cell to visualize the results.

In [ ]:
%use sos
# Declare the lists of images and titles for the display
images = [plate, butterfly, np.array(plate_erosion), np.array(butterfly_erosion)]
image_names = ['plate', 'butterfly', 'plate eroded', 'butterfly eroded']

# Display all 4 images
plt.close('all')
erosion_results = viewer(images, title=image_names, subplots=(2,2))

OpenCV has implemented the function [`cv.erode`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb), which we can use to erode an image with a custom structuring element. The main parameters are:
 * `src`: The original image, 
 * `kernel`: The structuring element, 
 * `borderType`: The boundary conditions. 
 
We encourage you to look at the [documentation](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb) for more details.
You might find some other interesting parameters, such as `iterations`. 
    
⚠️ **Notes: We use mirror padding here specifically because the `getNbh` method used in JavaScript applies this padding by default. This way we can correctly compare the two results, If the two images are not equal, you will see the differences in red (red areas mean that those pixels do not match). Make sure that you are using the same structuring elements! And use the info from the red areas to see where the differences might come from.**

It is best practice to explicitly declare arguments to Python functions when using image processing libraries -- except for the original image, as it is always the first one -- as we have been doing with `cv.getStructuringElement`. This is because the optional arguments are not necessarily in the order in which we will use them, and explicit declaration can avoid confusion (e.g. `cv.erode(img, kernel=b, borderType=cv.BORDER_REFLECT)` is not the same as `cv.erode(img, b, cv.BORDER_REFLECT)`! If you go through the documentation, you will see that there is the parameter `dst` in between them.

In [ ]:
%use sos
# Lets erode the plate image with OpenCV to see if the results are the same
# Define structuring element - feel free to change it, but it should be the same as in JavaScript for the comparison to make sense!
b = cv.getStructuringElement(cv.MORPH_RECT, ksize=(5,5))
# Erode plate
plate_erosion_cv = cv.erode(plate, kernel=b, borderType=cv.BORDER_REFLECT)

# Compare the two versions visually
plt.close('all')
erosion_comp = viewer([np.array(plate_erosion), plate_erosion_cv], title=['JS eroded plate', 'OpenCV eroded plate'], subplots=(1,2), compare=True)

In [ ]:
%use sos
# And numerically
assert np.allclose(plate_erosion, plate_erosion_cv), 'Sorry, but the eroded images are not identical. Make sure you used the same structuring elements for both functions.'
print('Nice! Your erosion function gives the same result as OpenCV on the plate image.')

## 3.B. Dilation

In the cell below, **for 1 point**, complete the code in JavaScript to implement the `dilation(img, b)`.

Remember that in the course, given an image $f$ with support $\Omega_f$ and a structuring element $b$ with support $\Omega_b$, we define dilation as

$$
    (f \oplus b)[\mathbf{k}] = \max_{\mathbf{q}\in\Omega_b}\left\lbrace f\left[\mathbf{k} - \mathbf{q}\right] \mid (\mathbf{k}-\mathbf{q})\in\Omega_f \right\rbrace\,.
$$

In [ ]:
%use javascript

// function that performs a dilation on the image 'img' using the structuring element 'b'
function dilation(img, b) {
    // declaring the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    
    return output;
}

// Here we declare the structuring element
var b = square(5); // Feel free to change it to your liking (using the functions implemented in part 1) and observe the results.

Now run the next two cells to dilate the images `plate` and `butterfly` and see the results. 

In [ ]:
%use javascript
%put plate_dilation
%put butterfly_dilation

// running the operation and converting the images back to python
var plate_dilation = dilation(new Image(plate), b).toArray();
var butterfly_dilation = dilation(new Image(butterfly), b).toArray();

In [ ]:
%use sos
# Define the image and title lists for the visualization
images = [plate, butterfly, np.array(plate_dilation), np.array(butterfly_dilation)]
image_names = ['plate', 'butterfly', 'plate dilated', 'butterfly dilated']

# Display the results
plt.close('all')
dilation_results = viewer(images, title=image_names, subplots=(2,2))

As we did for the erosion, in Python we can use [`cv.dilate`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#ga4ff0f3318642c4f469d0e11f242f3b6c) to dilate an image `img` with a structuring element `b`. The input arguments are the same as for `cv.erode`. Run the cells below to compare your implementation to the one of OpenCV.

In [ ]:
%use sos
# Lets dilate the plate image using OpenCV
# As always, we first define the structuring element we want to use
b = cv.getStructuringElement(cv.MORPH_RECT, ksize=(5,5))

# Then we perform the dilation
plate_dilation_cv = cv.dilate(plate, kernel=b, borderType=cv.BORDER_REFLECT)

# And compare the two versions visually
plt.close('all')
dilation_comp = viewer([np.array(plate_dilation), plate_dilation_cv], title=['JS dilated plate', 'OpenCV dilated plate'], subplots=(1,2), compare=True)

In [ ]:
%use sos
# And compare numerically
assert np.allclose(plate_dilation, plate_dilation_cv), 'Sorry, but the dilated images are not identical. Make sure you used the same structuring elements for both functions.'
print("That's it! Your dilation function produces the same result as OpenCV on the plate image.")

## 3.C. Median filter

In the cell below, **for 1 point**, complete the code in JavaScript to implement the `median()` filter.

Remember that in the course, given an image $f$ with support $\Omega_f$ and a structuring element $b$ with support $\Omega_b$, we define the median filter as

$$
    \mathrm{MED}(f, b)[\mathbf{k}] = \mathrm{median}\left( \left\lbrace f\left[\mathbf{k} - \mathbf{q}\right] \mid \mathbf{q}\in\Omega_b, (\mathbf{k}-\mathbf{q})\in\Omega_f \right\rbrace \right)\,.
$$

Here, $\mathrm{median}(\cdot)$ is a function that acts on a set of numbers, as usual in statistics (see, e.g., [here](https://en.wikipedia.org/wiki/Median#Finite_data_set_of_numbers)). These numbers are specified by the mathematical expression inside $\lbrace\cdot\rbrace$. 

💡 *Hint: You can use `nbh.sort(b)` to get a sorted (low to high) 1D `Image`object of the pixels in `nbh` that are under the `True` values of the structuring element `b` The sorted `Image` object consists of just one row, so the length of the sorted numbers is given by `sorted.nx`. Example: use `sorted.getPixel(2, 0)` to extract the 3rd smallest value.*

In [ ]:
%use javascript

// function that performs a median on the image 'img' using the structuring element 'b'
function median(img, b) {
    // declaring the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    
    return output;
}

// here we declare the structuring element
var b = square(5); // Feel free to change it to your liking (using the functions implemented in part 1) and observe the results.

In [ ]:
%use javascript
%put plate_median
%put butterfly_median

// running the operation and converting the images back to python
var plate_median = median(new Image(plate), b).toArray();
var butterfly_median = median(new Image(butterfly), b).toArray();

In [ ]:
%use sos
# Define the lists of names and images for visualization
images = [plate, butterfly, np.array(plate_median), np.array(butterfly_median)]
image_names = ['plate', 'butterfly', 'plate median', 'butterfly median']

# Display the results
plt.close('all')
median_results = viewer(images, title=image_names, subplots=(2,2))

To apply a median filter to an image `img` with an $n \times n$ **square** structuring element using OpenCV, we can use [`cv.medianBlur(img, ksize=n)`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#ga564869aa33e58769b4469101aac458f9). Run the cell below to compare your median to the OpenCV median.

**Note: In OpenCV, the median filter can only be applied with a square structuring element, and uses "repeat padding" at the border, i.e., it repeats the last pixel. That means that you just coded a more versatile median filter than the one from one of the most recognized image-processing libraries!**

In [ ]:
%use sos
# Lets apply the median to the plate image
# Set the size of the square structuring element
n = 5
# Run the operation
plate_median_cv = cv.medianBlur(plate, ksize=n)
# Compare the two versions visually
plt.close('all')
median_comp = viewer([np.array(plate_median), plate_median_cv], title=['JS median filtered plate', 'OpenCV median filtered plate'], subplots=(1,2), compare=True)

⚠️ **Note: Because the OpenCV median uses "repeat padding" instead of "mirror padding" as we do, we should not compare the border region of the images. To be sure, we leave the $\frac{n}{2} + 1$ outer-most rows and columns out of the comparison, where $n$ is the size of the square structuring element.**

In [ ]:
%use sos
# And numerically
assert np.allclose(np.array(plate_median)[n//2+1:np.shape(plate_median)[0]-n//2, n//2+1:np.shape(plate_median)[1]-n//2], 
                   plate_median_cv[n//2+1:plate_median_cv.shape[0]-n//2, n//2+1:plate_median_cv.shape[1]-n//2]), 'Sorry, but your median filter still needs some work.'
print("Very good! Your median filter gives the same result as OpenCV for the plate image.")

Now let's take a moment to look at the [morphological filters' table](#3.-Morphological-filters-(9-points)), given at the beginning of Part 3. There, you will see that all the morphological filters that we have not implemented yet are simple combinations of some of those that we have implemented, namely `dilation` and `erosion`. Since the idea of this lab is for you to understand how morphological filters work and not to write down an unnecessary amount of `for` loops in JavaScript, **we will now switch to only using Python and OpenCV for the rest of the lab**.

## 3.D. Opening

In the cell below, **for 1 point**, implement the `open()` function **using only the [`cv.dilate`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#ga4ff0f3318642c4f469d0e11f242f3b6c) and [`cv.erode`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb) functions presented above**.

⚠️ **Note: To be consistent, make sure you use `cv.BORDER_REFLECT` as the border type for all functions.**

In [ ]:
%use sos

# Function that performs an opening on the image 'img' using the structuring element 'b'
def opening(img, b):
    # Declaring the output image
    AoB = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return AoB

# Here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, ksize=(5,5)); # Feel free to change it to your liking (using the functions presented in part 1) and observe the results.

# Directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_open = opening(plate, b)
butterfly_open = opening(butterfly, b)

# and also display the result
# Define the lists of images and titles
images = [plate, butterfly, plate_open, butterfly_open]
image_names = ['plate', 'butterfly', 'plate opened', 'butterfly opened']
# Visualize them
plt.close('all')
open_results = viewer(images, title=image_names, subplots=(2,2))

Now let's compare your `open` function to the one that OpenCV provides. Using OpenCV, you can perform the opening of an image `img` with a structuring element `b` using [`cv.morphologyEx`](https://docs.opencv.org/4.5.2/d4/d86/group__imgproc__filter.html#ga67493776e3ad1a3df63883829375201f). Its main parameters are very similar to the other morphology related functions:
 * `src`: the original image, 
 * `op`: The operation to be performed (see OpenCV's [MorphTypes](https://docs.opencv.org/4.5.2/d4/d86/group__imgproc__filter.html#ga7be549266bad7b2e6a04db49827f9f32) list)
 * `kernel`: Structuring element to use, 
 * `borderType`: Boundary conditions.

Again we use `borderType=cv.BORDER_REFLECT` for consistency. Run the next cell to compare the functions.

⚠️ **Note: Of course, it is forbidden to use the function `cv.morphologyEx` inside your functions `opening`, `closing`, and `gradient`.**

In [ ]:
%use sos
# Perform the opening on plate with OpenCV using the same structuring element b
plate_open_cv = cv.morphologyEx(src=plate, op=cv.MORPH_OPEN, kernel=b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
if not np.allclose(plate_open, plate_open_cv): 
    print('WARNING!\nSorry, your opening does not match the opening of OpenCV.')
else :
    print("Great! Your opening gives the same result as OpenCV on the plate image.")

## 3.E. Closing

In the cell below, **for 1 point**, implement the `close()` function **using only the [`cv.dilate`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#ga4ff0f3318642c4f469d0e11f242f3b6c) and [`cv.erode`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb) functions presented above**.

In [ ]:
%use sos

# Function that performs a closing on the image 'img' using the structuring element 'b'
def closing(img, b):
    # Declaring the output image
    AcB = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return AcB

# Here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, ksize=(5,5)); # Feel free to change it to your liking (using the functions discussed and implemented in part 1) and observe the results.

# Directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_close = closing(plate, b)
butterfly_close = closing(butterfly, b)

# and also display the result
# Define the lists of images and titles
images = [plate, butterfly, plate_close, butterfly_close]
image_names = ['plate', 'butterfly', 'plate closed', 'butterfly closed']
# Visualize them
plt.close('all')
close_results = viewer(images, title=image_names, subplots=(2,2))

To perform the closing operation with OpenCV, we use `cv.MORPH_CLOSE` instead of `cv.MORPH_OPEN` in the `cv.morphologyEx` function presented above. Run the cell below to compare the functions.

In [ ]:
%use sos
# Perform the closing on plate with OpenCV using the same structuring element b
plate_close_cv = cv.morphologyEx(src=plate, op=cv.MORPH_CLOSE, kernel=b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
if not np.allclose(plate_close, plate_close_cv): 
    print('Sorry, your closing is not quite right.')
else:
    print("Great! Your closing gives the same result as OpenCV on the plate image.")

## 3.F. Gradient filter

In the cell below, **for 1 point**, implement the `gradient()` function **using only the [`cv.dilate`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#ga4ff0f3318642c4f469d0e11f242f3b6c) and [`cv.erode`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb) functions presented above** and basic arithmetics (`+`,`-`,`*`, or `/`).

💡 *Hint: Remember that in Python you can add/subtract whole images simply by using the $+/-$ operators, without the need to iterate with `for` loops.*

In [ ]:
%use sos

# function that performs a gradient on the image 'img' using the structuring element 'b'
def gradient(img, b):
    # declaring the output image
    grad = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return grad

# here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, ksize=(5,5)); # Feel free to change it to your liking (using the functions discussed and implemented in part 1) and observe the results.

# directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_gradient = gradient(plate, b)
butterfly_gradient = gradient(butterfly, b)

# and also display the result
# define the lists of images and titles
images = [plate, butterfly, plate_gradient, butterfly_gradient]
image_names = ['plate', 'butterfly', 'plate gradient', 'butterfly gradient']
# visualize them
plt.close('all')
gradient_results = viewer(images, title=image_names, subplots=(2,2))

In OpenCV, the identifier `cv.MORPH_GRADIENT` can be used to calculate the gradient with the `cv.morphologyEx` function presented above. Run the next cell to check your function.

In [ ]:
%use sos
# Perform the gradient on plate with OpenCV using the same structuring element b
plate_gradient_cv = cv.morphologyEx(src=plate, op=cv.MORPH_GRADIENT, kernel=b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
if not np.allclose(plate_gradient, plate_gradient_cv): 
    print('WARNING!\nSorry, your gradient is not quite right.')
else:
    print("Great! Your gradient operator produces the same result as OpenCV on the plate image.")

## 3.G. Top-hat filter

In the cell below, **for 1 point**, implement the `topHat()` function by **using only the OpenCV functions we have seen in Tasks [3.A.](#3.A.-Erosion) to [3.E.](#3.E.-Closing)**.

⚠️ **Note: You are not allowed to use `cv.morphologyEx` with `cv.MORPH_TOPHAT` in this function.**

In [ ]:
%use sos

# function that performs a topHat on the image 'img' using the structuring element 'b'
def topHat(img, b):
    # declaring the output image
    tophat = np.zeros(img.shape)
    
    # YOUR CODE HERE

    return tophat
    
# here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, ksize=(5, 5)); # Feel free to change it to your liking (using the functions discussed and implemented in part 1) and observe the results.

# directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_tophat = topHat(plate, b)
butterfly_tophat = topHat(butterfly, b)

# and also display the result
# define the lists of images and titles
images = [plate, butterfly, plate_tophat, butterfly_tophat]
image_names = ['plate', 'butterfly', 'plate tophat', 'butterfly tophat']
# visualize them
plt.close('all')
tophat_results = viewer(images, title=image_names, subplots=(2,2))

To perform the topHat filter in OpenCV, one uses `cv.MORPH_TOPHAT`. Run the cell below to check your function.

In [ ]:
%use sos
# Perform the top hat on plate with OpenCV using the same structuring element b
plate_tophat_cv = cv.morphologyEx(src=plate, op=cv.MORPH_TOPHAT, kernel=b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
if not np.allclose(plate_tophat, plate_tophat_cv): 
    print('Sorry, your topHat filter is not quite right.')
else :
    print("Great! Your topHat operator produces the same result as OpenCV on the plate image.")

## 3.H. Bottom-hat filter

In the cell below, **for 1 point**, implement the `bottomHat()` function by **using only the OpenCV functions we have seen in Tasks [3.A.](#3.A.-Erosion) to [3.E.](#3.E.-Closing)**.

⚠️ **Note: You are not allowed to use `cv.morphologyEx` with `op=cv.MORPH_BLACKHAT` in this function.**

In [ ]:
%use sos

# function that performs a bottomHat on the image 'img' using the structuring element 'b'
def bottomHat(img, b):
    # declaring the output image
    bottomhat = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return bottomhat

# here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, ksize=(5,5)); # Feel free to change it to your liking (using the functions discussed and implemented in part 1) and observe the results.

# directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_bottomhat = bottomHat(plate, b)
butterfly_bottomhat = bottomHat(butterfly, b)

# and also display the result
# define the lists of images and titles
images = [plate, butterfly, plate_bottomhat, butterfly_bottomhat]
image_names = ['plate', 'butterfly', 'plate bottomHat', 'butterfly bottomHat']
# visualize them
plt.close('all')
bottomhat_results = viewer(images, title=image_names, subplots=(2,2))

In OpenCV we use `cv.MORPH_BLACKHAT` to perform a bottomHat filter. Run the next cell to check your function.

In [ ]:
%use sos
# Perform the bottom hat on plate with OpenCV using the same structuring element b
plate_bottomhat_cv = cv.morphologyEx(plate, cv.MORPH_BLACKHAT, b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
if not np.allclose(plate_bottomhat, plate_bottomhat_cv): 
    print('Sorry, your bottomHat filter is not quite right.')
else:
    print("Great! Your bottomHat operator produces the same result as OpenCV on the plate image.")

## 3.I. Understanding morphological filters

Which of the following statements are correct? Here, $N\times$ Function(`name`, $y$) refers to composing a function $N$ times with the structuring element given by `name` with size $y\times y$.

1. The results of $3 \times$ Erosion(Square, 3) and Erosion(Square, 7) are the same.
2. The results of $3 \times$ Open(Cross, 5) and Open(Cross, 5) are the same.
3. The results of $3 \times$ Close(Disk, 5) and Close(Disk, 5) are the same.
4. The results of Top-Hat(Square, 3) and Bottom-Hat (Square, 3) are the same.

You can use the next cell to compare the different propositions by modifying the existing code and inserting your own.

⚠️ **Note:** 
- **You can use all the Python tools that you have seen previously to answer the questions.**
- **You will not be graded on the code below, but only on the answers.**
- **To get a disc structuring element you can use the `disc` function we provided in Part [1.C](#1.C.-Disc-structuring-element).**

In [ ]:
%use sos
# We use the butterfly image, feel free to use any other image
modified_1 = butterfly
modified_2 = butterfly

# Apply the morphological operators

# YOUR CODE HERE

# Display the two modified images as well as their difference
images = [modified_1, modified_2, modified_2 - modified_1]
titles = ['Operation 1', 'Operation 2', 'Difference']

plt.close('all')
operation_comparison = viewer(images, title=titles, subplots=(2,2))

**In the next cell, ***for 0.25 points*** per statement, assign `True` to the statements you think are correct and assign `False` to the statements you think are incorrect. The following cells are for you to check that your answers are valid.**

In [ ]:
%use sos
# Example: 'statement_0 = True' or 'statement_0 = False'
statement_1 = None
statement_2 = None
statement_3 = None
statement_4 = None

# YOUR CODE HERE

In [ ]:
%use sos
# Perform sanity check on statement_1
if not statement_1 in [True, False]: 
    print('WARNING!\nAssign either True or False to statement_1.')

In [ ]:
%use sos
# Perform sanity check on statement_2
if not statement_2 in [True, False]: 
    print('WARNING!\nAssign either True or False to statement_2.')

In [ ]:
%use sos
# Perform sanity check on statement_3
if not statement_3 in [True, False]:
    print('WARNING!\nAssign either True or False to statement_3.')

In [ ]:
%use sos
# Perform sanity check on statement_4
if not statement_4 in [True, False]: 
    print('WARNING!\nAssign either True or False to statement_4.')

🎉 Congratulations on finishing the first part of the Morphology lab! 🎉 Hopefully, you are now familiar with the basic tools needed to perform morphological operations. Your next step is to complete the second part of the lab, [Lab 3.2: Morphology Applications](./2_Morphology_Applications.ipynb), in which you will apply the basic tools you've learned in this part, to create interesting and useful image processing applications.

Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to [Moodle](https://moodle.epfl.ch/course/view.php?id=522), **in a zip file with other notebooks of this lab**.

* Please do not rename the notebook!
* Name the `zip` file *Morphology_Lab.zip*.